<a href="https://colab.research.google.com/github/Mai-Fakhry/Optical-character-recognition-OCR-/blob/main/NoteBooks/OCR_Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
import os
import zipfile
import shutil

In [4]:
!git clone https://github.com/Mai-Fakhry/Optical-character-recognition-OCR-

Cloning into 'Optical-character-recognition-OCR-'...
remote: Enumerating objects: 33, done.
remote: Counting objects: 100% (33/33), done.
remote: Compressing objects: 100% (26/26), done.
remote: Total 33 (delta 9), reused 15 (delta 2), pack-reused 0 (from 0)
Receiving objects: 100% (33/33), 87.92 KiB | 1.24 MiB/s, done.
Resolving deltas: 100% (9/9), done.


In [5]:
!pip install kaggle

In [6]:
!ls /content

jigsaw-puzzle-matching	Optical-character-recognition-OCR-  sample_data


In [9]:
!mkdir -p ~/.kaggle
!cp /content/kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [10]:
# Download and unzip
!kaggle datasets download -d harieh/ocr-dataset --unzip

Dataset URL: https://www.kaggle.com/datasets/harieh/ocr-dataset
License(s): CC0-1.0
 90% 137M/153M [00:00<00:00, 1.42GB/s]
100% 153M/153M [00:00<00:00, 1.29GB/s]


In [12]:
zip_path = "/content/Optical-character-recognition-OCR-/Datasets/template.zip"
extract_path = "/content"


os.makedirs(extract_path, exist_ok=True)

with zipfile.ZipFile(zip_path, 'r') as z:
    z.extractall(extract_path)

print("Extracted to:", extract_path)
print("Contents:", os.listdir(extract_path))

Extracted to: /content
Contents: ['.config', 'template', 'jigsaw-puzzle-matching', 'Optical-character-recognition-OCR-', 'kaggle.json', 'dataset', 'sample_data']


In [45]:
from PIL import Image
import numpy as np
import cv2
import os

In [56]:
# =========================
# 1. Define paths and letters
# =========================
template_dir = "/content/template/uppercase"  # templates folder
dataset_dir = "/content/dataset"              # dataset folder
letters = ['A', 'B', 'C', 'D','E']


In [57]:
# =========================
# 2. Preprocessing function: binarize, crop, center, resize, normalize
# =========================
def preprocess_image(img, size=(40, 60)):
    # Convert to grayscale if needed
    if len(img.shape) == 3:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # Binarize
    _, img_bin = cv2.threshold(img, 128, 255, cv2.THRESH_BINARY_INV)
    # Find bounding box of the character
    coords = cv2.findNonZero(img_bin)
    x, y, w, h = cv2.boundingRect(coords)
    cropped = img_bin[y:y+h, x:x+w]
    # Resize to fixed size
    resized = cv2.resize(cropped, size)
    # Normalize
    return resized / 255.0

In [58]:
# =========================
# 3. Load templates and flatten
# =========================
templates = {}
for char in letters:
    path = os.path.join(template_dir, f"{char}.png")
    img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    templates[char] = preprocess_image(img).flatten()

In [59]:
# =========================
# 4. Euclidean distance function
# =========================
def euclidean_distance(v1, v2):
    return np.sqrt(np.sum((v1 - v2)**2))

In [60]:
# =========================
# 5. Classification function
# =========================
def classify_single_char(test_vector, templates):
    min_dist = float('inf')
    predicted = None
    for char, vec in templates.items():
        dist = euclidean_distance(test_vector, vec)
        if dist < min_dist:
            min_dist = dist
            predicted = char
    return predicted

In [62]:
# =========================
# 6. Test on multiple letters
# =========================
test_images = [
    "/content/dataset/A_U/A_U_1042.png",
    "/content/dataset/B_U/B_U_1001.png",
    "/content/dataset/C_U/C_U_1000.png",
    "/content/dataset/E_U/E_U_1003.png"
]

for test_path in test_images:
    img = cv2.imread(test_path, cv2.IMREAD_GRAYSCALE)
    test_vector = preprocess_image(img).flatten()

    predicted_char = classify_single_char(test_vector, templates)
    print(f"Test image: {os.path.basename(test_path)} → Predicted Character: {predicted_char}")

Test image: A_U_1042.png → Predicted Character: A
Test image: B_U_1001.png → Predicted Character: B
Test image: C_U_1000.png → Predicted Character: C
Test image: E_U_1003.png → Predicted Character: E
